<a href="https://colab.research.google.com/github/camilla32/Assistente-virtual/blob/main/Agente_financeiro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
!pip install gradio -q

import gradio as gr
import re
import base64

# =========================
# AVATAR
# =========================
avatar_svg = """
<svg xmlns="http://www.w3.org/2000/svg" width="60" height="60">
  <rect width="60" height="60" rx="15" fill="#00f2ff"/>
  <text x="50%" y="55%" dominant-baseline="middle" text-anchor="middle"
        font-size="28" fill="#050a14" font-family="Arial">S</text>
</svg>
"""
avatar_base64 = base64.b64encode(avatar_svg.encode()).decode()

# =========================
# CSS ESTILO FINTECH
# =========================
css = """
.gradio-container {
    background-color: #050a14 !important;
    color: white !important;
}
.gradio-container * {
    color: white !important;
}
.saldi-chat .message.user {
    background: linear-gradient(90deg, #0077b6, #00d4ff) !important;
    border-radius: 20px !important;
}
.saldi-chat .message.bot {
    background-color: #1b263b !important;
    border-radius: 20px !important;
}
#msg_input textarea {
    background-color: #0d1b2a !important;
    border: 1px solid #00f2ff !important;
    border-radius: 20px !important;
    color: white !important;
}
"""

# =========================
# ESTADO GLOBAL
# =========================
estado_usuario = {
    "intencao": None,
    "meta_nome": None,
    "meta_valor": None,
    "meta_prazo": None,
    "aguardando_confirmacao": None
}

saldo_usuario = 3500  # saldo simulado

# =========================
# FUNÇÕES AUXILIARES
# =========================
def normalizar(texto):
    texto = texto.lower()
    texto = re.sub(r'[^\w\s]', '', texto)
    return texto

def detectar_intencao(msg):
    if any(p in msg for p in ["saldo", "quanto tenho", "meu saldo"]):
        return "saldo"
    if any(p in msg for p in ["meta", "planejar", "objetivo", "guardar", "economizar"]):
        return "planejamento"
    if "score" in msg:
        return "score"
    if any(p in msg for p in ["risco", "divida"]):
        return "risco"
    return None

# =========================
# LÓGICA PRINCIPAL
# =========================
def responder(mensagem, historico):

    global estado_usuario, saldo_usuario
    msg = normalizar(mensagem)

    if historico is None:
        historico = []

    resposta = ""

    # =========================
    # TRATAR CONFIRMAÇÃO
    # =========================
    if estado_usuario["aguardando_confirmacao"]:

        if "sim" in msg:
            acao = estado_usuario["aguardando_confirmacao"]
            estado_usuario["aguardando_confirmacao"] = None
            estado_usuario["intencao"] = acao

        elif "nao" in msg or "não" in msg:
            estado_usuario["aguardando_confirmacao"] = None
            resposta = "Tudo bem 💙 Estou aqui se precisar."
            historico.append((mensagem, resposta))
            return historico, ""

    # =========================
    # DETECTAR INTENÇÃO
    # =========================
    if not estado_usuario["intencao"]:
        estado_usuario["intencao"] = detectar_intencao(msg)

    # =========================
    # CONSULTA DE SALDO
    # =========================
    if estado_usuario["intencao"] == "saldo":
        resposta = f"""
💰 Seu saldo atual é:

R$ {saldo_usuario}

Deseja fazer um planejamento com base nesse valor?
"""
        estado_usuario["intencao"] = None
        estado_usuario["aguardando_confirmacao"] = "planejamento"

    # =========================
    # PLANEJAMENTO DE META
    # =========================
    elif estado_usuario["intencao"] == "planejamento":

        if not estado_usuario["meta_nome"]:
            estado_usuario["meta_nome"] = mensagem
            resposta = "Qual o valor necessário para essa meta?"

        elif not estado_usuario["meta_valor"]:
            numeros = re.findall(r'\d+', msg)
            if numeros:
                estado_usuario["meta_valor"] = int(numeros[0])
                resposta = "Em quantos meses deseja alcançar?"
            else:
                resposta = "Informe o valor em números."

        elif not estado_usuario["meta_prazo"]:
            numeros = re.findall(r'\d+', msg)
            if numeros:
                estado_usuario["meta_prazo"] = int(numeros[0])
                valor = estado_usuario["meta_valor"]
                prazo = estado_usuario["meta_prazo"]
                parcela = round(valor / prazo, 2)

                resposta = f"""
📊 Plano Criado

Meta: {estado_usuario['meta_nome']}
Valor: R$ {valor}
Prazo: {prazo} meses

Você precisa guardar:
R$ {parcela} por mês

Deseja que eu acompanhe essa meta?
"""
                estado_usuario["aguardando_confirmacao"] = "acompanhamento"

                estado_usuario["meta_nome"] = None
                estado_usuario["meta_valor"] = None
                estado_usuario["meta_prazo"] = None
                estado_usuario["intencao"] = None
            else:
                resposta = "Informe o prazo em meses."

    # =========================
    # SCORE
    # =========================
    elif estado_usuario["intencao"] == "score":
        resposta = """
📈 Seu Score estimado é 84/100.

Deseja dicas para melhorar?
"""
        estado_usuario["aguardando_confirmacao"] = "dicas_score"
        estado_usuario["intencao"] = None

    elif estado_usuario["intencao"] == "dicas_score":
        resposta = """
📘 Dicas para melhorar seu score:

• Pague contas antes do vencimento
• Reduza uso do crédito
• Mantenha reserva de emergência
"""
        estado_usuario["intencao"] = None

    # =========================
    # RISCO FINANCEIRO
    # =========================
    elif estado_usuario["intencao"] == "risco":
        resposta = """
⚠️ Detectei possível risco financeiro.

Deseja montar um plano de reorganização?
"""
        estado_usuario["aguardando_confirmacao"] = "plano_risco"
        estado_usuario["intencao"] = None

    elif estado_usuario["intencao"] == "plano_risco":
        resposta = """
🛠 Plano de Recuperação:

1. Liste todas as dívidas
2. Priorize maiores juros
3. Negocie parcelas
4. Defina plano de pagamento mensal
"""
        estado_usuario["intencao"] = None

    # =========================
    # ACOMPANHAMENTO
    # =========================
    elif estado_usuario["intencao"] == "acompanhamento":
        resposta = "Perfeito 💙 Vou acompanhar sua meta mensalmente."
        estado_usuario["intencao"] = None

    # =========================
    # DEFAULT
    # =========================
    else:
        resposta = """
Olá, eu sou o Saldi 💙

Posso ajudar com:

• Consulta de saldo
• Planejamento de metas
• Score financeiro
• Detecção de risco

Como posso ajudar?
"""

    historico.append((mensagem, resposta))
    return historico, ""

# =========================
# INTERFACE
# =========================
with gr.Blocks(css=css) as demo:

    gr.HTML(f"""
    <div style="display:flex; align-items:center; gap:15px; padding:20px 0;">
        <div style="width:60px; height:60px; border:2px solid #00f2ff;
        border-radius:15px; display:flex; align-items:center; justify-content:center;">
            <img src="data:image/svg+xml;base64,{avatar_base64}" width="45">
        </div>
        <div>
            <h2 style="margin:0;">Saldi <span style="color:#00f2ff;">Agente</span></h2>
            <p style="margin:0; font-size:13px;">
            Assistente Financeiro Inteligente</p>
        </div>
    </div>
    """)

    chatbot = gr.Chatbot(elem_classes="saldi-chat")
    msg = gr.Textbox(placeholder="Digite sua mensagem...", elem_id="msg_input")
    clear = gr.Button("Limpar Conversa")

    msg.submit(responder, [msg, chatbot], [chatbot, msg])
    clear.click(lambda: [], None, chatbot, queue=False)

demo.launch()

/tmp/ipython-input-1970084227.py:248: DeprecationWarning: The 'css' parameter in the Blocks constructor will be removed in Gradio 6.0. You will need to pass 'css' to Blocks.launch() instead.
  with gr.Blocks(css=css) as demo:
/tmp/ipython-input-1970084227.py:264: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(elem_classes="saldi-chat")
/tmp/ipython-input-1970084227.py:264: DeprecationWarning: The default value of 'allow_tags' in gr.Chatbot will be changed from False to True in Gradio 6.0. You will need to explicitly set allow_tags=False if you want to disable tags in your chatbot.
  chatbot = gr.Chatbot(elem_classes="saldi-chat")


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://03ef30c068b0055dec.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
